<a href="https://colab.research.google.com/github/alemolteni/codecarbon_project/blob/main/4_0_Stream_Ensemble_Classification_codecarbon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements

|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain

**Samples:** 18,159


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!cp "/gdrive/My Drive/CodeCarbon/datasets/NEweather.csv" /content
!cp "/gdrive/My Drive/CodeCarbon/datasets/agr_a_20k.csv" /content

Mounted at /gdrive


In [ ]:
!pip install river==0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for river: filename=river-0.7.0-cp310-cp310-linux_x86_64.whl size=2359732 sha256=e4758c1d4cdd2354485978c2211df7558b2d1b4c60359f8909f6718fd4834963
  Stored in directory: /root/.cache/pip/wheels/71/e9/7e/105173d51ebb5262f6f2dc4f6a5003ec86365255e8fd989733
Successfully built river


In [ ]:
!pip install codecarbon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.evaluate import progressive_val_score
from codecarbon import EmissionsTracker

In [ ]:
data = pd.read_csv("NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [ ]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

tracker = EmissionsTracker()
tracker.start()

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:21:55] [setup] RAM Tracking...
[codecarbon INFO @ 20:21:55] [setup] GPU Tracking...
[codecarbon INFO @ 20:21:55] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:21:55] [setup] CPU Tracking...
[codecarbon WARNING @ 20:21:55] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:21:57] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:21:57] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:21:57] >>> Tracker's metadata:
[codecarbon INFO @ 20:21:57]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:21:57]   Python version: 3.10.12
[codecarbon INFO @ 20:21:57]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:21:57]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:21:57]   CPU count: 2
[codecarbon INFO @ 20:21:57]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:21:57]   GPU

[1,000] Accuracy: 71.17%
[2,000] Accuracy: 70.09%
[3,000] Accuracy: 70.89%
[4,000] Accuracy: 71.42%
[5,000] Accuracy: 72.07%
[6,000] Accuracy: 72.51%


[codecarbon INFO @ 20:22:12] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:22:12] Energy consumed for all GPUs : 0.000041 kWh. Total GPU Power : 9.662 W
[codecarbon INFO @ 20:22:12] Energy consumed for all CPUs : 0.000180 kWh. Total CPU Power : 42.5 W


[7,000] Accuracy: 73.38%


[codecarbon INFO @ 20:22:13] 0.000240 kWh of electricity used since the beginning.


[8,000] Accuracy: 73.27%
[9,000] Accuracy: 73.36%
[10,000] Accuracy: 73.52%
[11,000] Accuracy: 73.99%
[12,000] Accuracy: 74.22%
[13,000] Accuracy: 74.31%
[14,000] Accuracy: 74.28%
[15,000] Accuracy: 74.54%
[16,000] Accuracy: 74.55%
[17,000] Accuracy: 74.62%


[codecarbon INFO @ 20:22:28] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:22:28] Energy consumed for all GPUs : 0.000081 kWh. Total GPU Power : 9.86 W
[codecarbon INFO @ 20:22:28] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:22:28] 0.000475 kWh of electricity used since the beginning.


[18,000] Accuracy: 74.38%


[codecarbon INFO @ 20:22:29] Energy consumed for RAM : 0.000042 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:22:29] Energy consumed for all GPUs : 0.000086 kWh. Total GPU Power : 9.86 W
[codecarbon INFO @ 20:22:29] Energy consumed for all CPUs : 0.000374 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:22:29] 0.000502 kWh of electricity used since the beginning.


6.960797401530874e-05

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [ ]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

tracker = EmissionsTracker()
tracker.start()

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:22:29] [setup] RAM Tracking...
[codecarbon INFO @ 20:22:29] [setup] GPU Tracking...
[codecarbon INFO @ 20:22:29] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:22:29] [setup] CPU Tracking...
[codecarbon WARNING @ 20:22:29] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:22:31] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:22:31] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:22:31] >>> Tracker's metadata:
[codecarbon INFO @ 20:22:31]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:22:31]   Python version: 3.10.12
[codecarbon INFO @ 20:22:31]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:22:31]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:22:31]   CPU count: 2
[codecarbon INFO @ 20:22:31]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:22:31]   GPU

[1,000] Accuracy: 75.78%
[2,000] Accuracy: 76.64%


[codecarbon INFO @ 20:22:46] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:22:46] Energy consumed for all GPUs : 0.000040 kWh. Total GPU Power : 9.662 W
[codecarbon INFO @ 20:22:47] Energy consumed for all CPUs : 0.000180 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:22:47] 0.000240 kWh of electricity used since the beginning.


[3,000] Accuracy: 77.79%
[4,000] Accuracy: 77.79%


[codecarbon INFO @ 20:23:01] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:23:01] Energy consumed for all GPUs : 0.000080 kWh. Total GPU Power : 9.562 W
[codecarbon INFO @ 20:23:01] Energy consumed for all CPUs : 0.000356 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:23:01] 0.000475 kWh of electricity used since the beginning.


...

[14,000] Accuracy: 78.16%
[15,000] Accuracy: 78.26%
[16,000] Accuracy: 78.27%


[codecarbon INFO @ 20:24:16] Energy consumed for RAM : 0.000137 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:24:16] Energy consumed for all GPUs : 0.000278 kWh. Total GPU Power : 9.761000000000001 W
[codecarbon INFO @ 20:24:17] Energy consumed for all CPUs : 0.001239 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:24:17] 0.001654 kWh of electricity used since the beginning.


[17,000] Accuracy: 78.26%
[18,000] Accuracy: 78.03%


[codecarbon INFO @ 20:24:30] Energy consumed for RAM : 0.000155 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:24:30] Energy consumed for all GPUs : 0.000314 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:24:30] Energy consumed for all CPUs : 0.001400 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:24:30] 0.001869 kWh of electricity used since the beginning.


0.0002593154528424729

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [ ]:
from river.ensemble import AdaptiveRandomForestClassifier

tracker = EmissionsTracker()
tracker.start()

model = AdaptiveRandomForestClassifier(n_models=10)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:24:30] [setup] RAM Tracking...
[codecarbon INFO @ 20:24:30] [setup] GPU Tracking...
[codecarbon INFO @ 20:24:30] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:24:30] [setup] CPU Tracking...
[codecarbon WARNING @ 20:24:30] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:24:32] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:24:32] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:24:32] >>> Tracker's metadata:
[codecarbon INFO @ 20:24:32]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:24:32]   Python version: 3.10.12
[codecarbon INFO @ 20:24:32]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:24:32]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:24:32]   CPU count: 2
[codecarbon INFO @ 20:24:32]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:24:32]   GPU

[1,000] Accuracy: 73.37%
[2,000] Accuracy: 76.09%
[3,000] Accuracy: 77.36%
[4,000] Accuracy: 77.22%


[codecarbon INFO @ 20:24:48] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:24:48] Energy consumed for all GPUs : 0.000043 kWh. Total GPU Power : 9.761000000000001 W
[codecarbon INFO @ 20:24:48] Energy consumed for all CPUs : 0.000194 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:24:48] 0.000257 kWh of electricity used since the beginning.


[5,000] Accuracy: 77.36%
[6,000] Accuracy: 77.38%
[7,000] Accuracy: 77.95%
[8,000] Accuracy: 77.75%


[codecarbon INFO @ 20:25:02] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:25:03] Energy consumed for all GPUs : 0.000079 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:25:03] Energy consumed for all CPUs : 0.000366 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:03] 0.000484 kWh of electricity used since the beginning.


...

[13,000] Accuracy: 78.44%
[14,000] Accuracy: 78.25%
[15,000] Accuracy: 78.46%


[codecarbon INFO @ 20:25:32] Energy consumed for RAM : 0.000076 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:25:32] Energy consumed for all GPUs : 0.000155 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:25:32] Energy consumed for all CPUs : 0.000712 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:32] 0.000944 kWh of electricity used since the beginning.


[16,000] Accuracy: 78.49%
[17,000] Accuracy: 78.49%
[18,000] Accuracy: 78.16%


[codecarbon INFO @ 20:25:44] Energy consumed for RAM : 0.000092 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:25:44] Energy consumed for all GPUs : 0.000187 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:25:44] Energy consumed for all CPUs : 0.000853 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:44] 0.001132 kWh of electricity used since the beginning.


0.00015704029717714653

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [ ]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

tracker = EmissionsTracker()
tracker.start()

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:25:44] [setup] RAM Tracking...
[codecarbon INFO @ 20:25:44] [setup] GPU Tracking...
[codecarbon INFO @ 20:25:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:25:44] [setup] CPU Tracking...
[codecarbon WARNING @ 20:25:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:25:46] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:25:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:25:46] >>> Tracker's metadata:
[codecarbon INFO @ 20:25:46]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:25:46]   Python version: 3.10.12
[codecarbon INFO @ 20:25:46]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:25:46]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:25:46]   CPU count: 2
[codecarbon INFO @ 20:25:46]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:25:46]   GPU

[1,000] Accuracy: 73.57%
[2,000] Accuracy: 76.29%


[codecarbon INFO @ 20:26:01] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:26:01] Energy consumed for all GPUs : 0.000040 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:26:02] Energy consumed for all CPUs : 0.000179 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:26:02] 0.000238 kWh of electricity used since the beginning.


[3,000] Accuracy: 77.79%
[4,000] Accuracy: 78.12%


[codecarbon INFO @ 20:26:16] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:26:16] Energy consumed for all GPUs : 0.000079 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:26:16] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:26:17] 0.000473 kWh of electricity used since the beginning.


...

[15,000] Accuracy: 78.67%
[16,000] Accuracy: 78.69%


[codecarbon INFO @ 20:27:50] Energy consumed for RAM : 0.000155 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:27:50] Energy consumed for all GPUs : 0.000315 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:27:50] Energy consumed for all CPUs : 0.001440 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:27:50] 0.001911 kWh of electricity used since the beginning.


[17,000] Accuracy: 78.74%
[18,000] Accuracy: 78.53%


[codecarbon INFO @ 20:28:02] Energy consumed for RAM : 0.000171 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:28:02] Energy consumed for all GPUs : 0.000346 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:28:02] Energy consumed for all CPUs : 0.001579 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:28:02] 0.002096 kWh of electricity used since the beginning.


0.0002908402483661201

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan

**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [ ]:
data = pd.read_csv("agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [ ]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

tracker = EmissionsTracker()
tracker.start()

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:28:02] [setup] RAM Tracking...
[codecarbon INFO @ 20:28:02] [setup] GPU Tracking...
[codecarbon INFO @ 20:28:02] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:28:02] [setup] CPU Tracking...
[codecarbon WARNING @ 20:28:02] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:28:04] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:28:04] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:28:04] >>> Tracker's metadata:
[codecarbon INFO @ 20:28:04]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:28:04]   Python version: 3.10.12
[codecarbon INFO @ 20:28:04]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:28:04]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:28:04]   CPU count: 2
[codecarbon INFO @ 20:28:04]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:28:04]   GPU

[1,000] Accuracy: 80.18%
[2,000] Accuracy: 84.04%
[3,000] Accuracy: 85.46%
[4,000] Accuracy: 86.70%
[5,000] Accuracy: 87.62%
[6,000] Accuracy: 82.60%
[7,000] Accuracy: 80.57%


[codecarbon INFO @ 20:28:19] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:28:19] Energy consumed for all GPUs : 0.000039 kWh. Total GPU Power : 9.364 W


[8,000] Accuracy: 79.58%


[codecarbon INFO @ 20:28:19] Energy consumed for all CPUs : 0.000183 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:28:20] 0.000242 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 20:28:34] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:28:35] Energy consumed for all GPUs : 0.000077 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:28:35] Energy consumed for all CPUs : 0.000360 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:28:35] 0.000476 kWh of electricity used since the beginning.


[16,000] Accuracy: 76.69%
[17,000] Accuracy: 76.31%
[18,000] Accuracy: 76.25%
[19,000] Accuracy: 76.00%


[codecarbon INFO @ 20:28:44] Energy consumed for RAM : 0.000051 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:28:44] Energy consumed for all GPUs : 0.000101 kWh. Total GPU Power : 9.562 W
[codecarbon INFO @ 20:28:44] Energy consumed for all CPUs : 0.000466 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:28:44] 0.000618 kWh of electricity used since the beginning.


[20,000] Accuracy: 75.77%


8.576070118618925e-05

## Leveraging Bagging

In [ ]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

tracker = EmissionsTracker()
tracker.start()

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:28:44] [setup] RAM Tracking...
[codecarbon INFO @ 20:28:44] [setup] GPU Tracking...
[codecarbon INFO @ 20:28:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:28:44] [setup] CPU Tracking...
[codecarbon WARNING @ 20:28:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:28:45] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:28:45] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:28:45] >>> Tracker's metadata:
[codecarbon INFO @ 20:28:45]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:28:45]   Python version: 3.10.12
[codecarbon INFO @ 20:28:45]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:28:45]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:28:45]   CPU count: 2
[codecarbon INFO @ 20:28:45]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:28:45]   GPU

[1,000] Accuracy: 75.48%
[2,000] Accuracy: 79.34%


[codecarbon INFO @ 20:29:00] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:29:01] Energy consumed for all GPUs : 0.000041 kWh. Total GPU Power : 9.761000000000001 W
[codecarbon INFO @ 20:29:01] Energy consumed for all CPUs : 0.000180 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:29:01] 0.000241 kWh of electricity used since the beginning.


[3,000] Accuracy: 81.83%
[4,000] Accuracy: 83.45%
[5,000] Accuracy: 84.56%


[codecarbon INFO @ 20:29:15] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:29:15] Energy consumed for all GPUs : 0.000083 kWh. Total GPU Power : 10.158000000000001 W
[codecarbon INFO @ 20:29:16] Energy consumed for all CPUs : 0.000356 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:29:16] 0.000478 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 20:30:45] Energy consumed for RAM : 0.000157 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:30:45] Energy consumed for all GPUs : 0.000322 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:30:46] Energy consumed for all CPUs : 0.001416 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:30:46] 0.001895 kWh of electricity used since the beginning.


[19,000] Accuracy: 74.52%


[codecarbon INFO @ 20:30:52] Energy consumed for RAM : 0.000166 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:30:52] Energy consumed for all GPUs : 0.000338 kWh. Total GPU Power : 9.364 W
[codecarbon INFO @ 20:30:52] Energy consumed for all CPUs : 0.001492 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:30:52] 0.001996 kWh of electricity used since the beginning.


[20,000] Accuracy: 74.23%


0.00027691706811611524

## AdaptiveRandomForest

In [ ]:
from river.ensemble import AdaptiveRandomForestClassifier

tracker = EmissionsTracker()
tracker.start()

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:30:52] [setup] RAM Tracking...
[codecarbon INFO @ 20:30:52] [setup] GPU Tracking...
[codecarbon INFO @ 20:30:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:30:52] [setup] CPU Tracking...
[codecarbon WARNING @ 20:30:52] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:30:53] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:30:53] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:30:53] >>> Tracker's metadata:
[codecarbon INFO @ 20:30:53]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:30:53]   Python version: 3.10.12
[codecarbon INFO @ 20:30:53]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:30:53]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:30:53]   CPU count: 2
[codecarbon INFO @ 20:30:53]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:30:53]   GPU

[1,000] Accuracy: 68.37%
[2,000] Accuracy: 70.14%
[3,000] Accuracy: 71.46%
[4,000] Accuracy: 72.69%


[codecarbon INFO @ 20:31:09] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:31:09] Energy consumed for all GPUs : 0.000040 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:31:09] Energy consumed for all CPUs : 0.000185 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:31:09] 0.000244 kWh of electricity used since the beginning.


[5,000] Accuracy: 73.25%
[6,000] Accuracy: 69.56%
[7,000] Accuracy: 68.50%
[8,000] Accuracy: 67.18%


[codecarbon INFO @ 20:31:24] Energy consumed for RAM : 0.000039 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:31:24] Energy consumed for all GPUs : 0.000078 kWh. Total GPU Power : 9.364 W
[codecarbon INFO @ 20:31:24] Energy consumed for all CPUs : 0.000362 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:31:24] 0.000478 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 20:31:54] Energy consumed for RAM : 0.000078 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:31:54] Energy consumed for all GPUs : 0.000154 kWh. Total GPU Power : 9.364 W
[codecarbon INFO @ 20:31:54] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:31:54] 0.000941 kWh of electricity used since the beginning.


[17,000] Accuracy: 65.50%
[18,000] Accuracy: 65.88%
[19,000] Accuracy: 66.25%


[codecarbon INFO @ 20:32:09] Energy consumed for RAM : 0.000097 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:32:09] Energy consumed for all GPUs : 0.000194 kWh. Total GPU Power : 9.562 W
[codecarbon INFO @ 20:32:09] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:32:09] 0.001177 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:32:10] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:32:10] Energy consumed for all GPUs : 0.000196 kWh. Total GPU Power : 9.662 W
[codecarbon INFO @ 20:32:10] Energy consumed for all CPUs : 0.000897 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:32:10] 0.001191 kWh of electricity used since the beginning.


[20,000] Accuracy: 66.56%


0.00016528594823228071

## StreamingRandomPatches
---
We set the drift and warning detection options

In [ ]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

tracker = EmissionsTracker()
tracker.start()

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)

metrics = Metrics(metrics=[Accuracy()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

tracker.stop()

[codecarbon INFO @ 20:32:10] [setup] RAM Tracking...
[codecarbon INFO @ 20:32:10] [setup] GPU Tracking...
[codecarbon INFO @ 20:32:10] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:32:10] [setup] CPU Tracking...
[codecarbon WARNING @ 20:32:10] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:32:11] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:32:11] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:32:11] >>> Tracker's metadata:
[codecarbon INFO @ 20:32:11]   Platform system: Linux-5.15.107+-x86_64-with-glibc2.31
[codecarbon INFO @ 20:32:11]   Python version: 3.10.12
[codecarbon INFO @ 20:32:11]   CodeCarbon version: 2.2.4
[codecarbon INFO @ 20:32:11]   Available RAM : 12.678 GB
[codecarbon INFO @ 20:32:11]   CPU count: 2
[codecarbon INFO @ 20:32:11]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 20:32:11]   GPU

[1,000] Accuracy: 90.59%
[2,000] Accuracy: 92.35%


[codecarbon INFO @ 20:32:28] Energy consumed for RAM : 0.000021 kWh. RAM Power : 4.754399299621582 W


[3,000] Accuracy: 92.90%


[codecarbon INFO @ 20:32:29] Energy consumed for all GPUs : 0.000044 kWh. Total GPU Power : 9.562 W
[codecarbon INFO @ 20:32:29] Energy consumed for all CPUs : 0.000212 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:32:29] 0.000278 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 20:34:01] Energy consumed for RAM : 0.000129 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:34:01] Energy consumed for all GPUs : 0.000265 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:34:01] Energy consumed for all CPUs : 0.001253 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:01] 0.001647 kWh of electricity used since the beginning.


[19,000] Accuracy: 80.89%


[codecarbon INFO @ 20:34:12] Energy consumed for RAM : 0.000144 kWh. RAM Power : 4.754399299621582 W
[codecarbon INFO @ 20:34:12] Energy consumed for all GPUs : 0.000293 kWh. Total GPU Power : 9.463000000000001 W
[codecarbon INFO @ 20:34:12] Energy consumed for all CPUs : 0.001380 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:12] 0.001817 kWh of electricity used since the beginning.


[20,000] Accuracy: 80.53%


0.0002521062220249356